In [3]:
import heapq
from collections import defaultdict

def parse_map_from_file(file_path):
    with open(file_path, 'r') as f:
        racetrack = f.read().strip().split("\n")

    start, end = None, None
    grid = {}

    for y, row in enumerate(racetrack):
        for x, cell in enumerate(row):
            grid[(x, y)] = cell
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)

    return grid, start, end

def dijkstra(grid, start, end, allow_cheat=False):
    heap = [(0, start, False)]  # (distance, position, used_cheat)
    distances = {(start, False): 0}

    while heap:
        distance, current, cheated = heapq.heappop(heap)

        if current == end:
            return distance

        if distances.get((current, cheated), float('inf')) < distance:
            continue

        x, y = current
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = (x + dx, y + dy)

            if neighbor in grid and grid[neighbor] in {'.', 'S', 'E'}:
                new_distance = distance + 1
                if new_distance < distances.get((neighbor, cheated), float('inf')):
                    distances[(neighbor, cheated)] = new_distance
                    heapq.heappush(heap, (new_distance, neighbor, cheated))

            elif allow_cheat and not cheated and neighbor in grid and grid[neighbor] == '#':
                # Use cheat to pass through wall
                new_distance = distance + 1
                if new_distance < distances.get((neighbor, True), float('inf')):
                    distances[(neighbor, True)] = new_distance
                    heapq.heappush(heap, (new_distance, neighbor, True))

    return float('inf')

def find_all_cheats(grid, start, end):
    cheats = defaultdict(list)
    shortest_no_cheat = dijkstra(grid, start, end, allow_cheat=False)

    for x, y in grid:
        if grid[(x, y)] != '.':
            continue

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            cheat_start = (x, y)
            cheat_end = (x + 2 * dx, y + 2 * dy)

            if cheat_end in grid and grid[cheat_end] == '.':
                grid_copy = grid.copy()
                grid_copy[(x + dx, y + dy)] = '.'  # Temporarily clear wall
                cheat_path_length = dijkstra(grid_copy, start, end, allow_cheat=True)
                savings = shortest_no_cheat - cheat_path_length

                if savings > 0:
                    cheats[savings].append((cheat_start, cheat_end))

    return cheats

def count_large_savings_cheats(cheats, threshold):
    return sum(len(cheats[savings]) for savings in cheats if savings >= threshold)

# Load racetrack from file
file_path = '/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_20/input_20_12_2024.txt'
grid, start, end = parse_map_from_file(file_path)
cheats = find_all_cheats(grid, start, end)
result = count_large_savings_cheats(cheats, 100)
print("Number of cheats saving at least 100 picoseconds:", result)


Number of cheats saving at least 100 picoseconds: 26324


In [7]:
input=[]
file=open('/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_20/input_20_12_2024.txt')
startx, starty = 0,0
endx, endy = 0,0
with file as f:
    for line in f:
        line = line.strip()
        row=[]
        for e in line:
            row.append(e)
        input.append(row)
for i in range(len(input)):
    for j in range(len(input[0])):
        if input[i][j] == 'S':
            startx,starty=i,j
        if input[i][j] == 'E':
            endx,endy=i,j
def draw(input):
    for row in input:
        print(''.join(row))


draw(input)

directions=[(0,1),(1,0),(0,-1),(-1,0)]
from collections import defaultdict
import heapq
def dijkstra(input):
    global startx,starty,endx,endy
    dist=defaultdict(lambda : float('inf'))
    dist[startx,starty]=0
    parent={}
    que=[]
    heapq.heappush(que,(0,startx,starty))

    while que:
        d,ci,cj=heapq.heappop(que)
        for dir in directions:
            i,j=dir
            i,j=ci+i,cj+j
            if not (0 <= i < len(input) and 0 <= j < len(input[0])) or input[i][j] == '#':
                continue
            if d + 1 < dist[i,j]:
                dist[i,j] = d + 1
                parent[i,j]=(ci,cj)
                heapq.heappush(que,(dist[i,j],i,j))

    stack=[(endx,endy)]
    visited=set()
    while stack:
        x,y=stack.pop()
        if (x,y) in visited:
            continue
        visited.add((x,y))
        if (x,y) not in parent:
            break
        stack.append(parent[x,y])
    return visited,dist

#print(dijkstra(copy.deepcopy(input)))
orginal_path,dist=dijkstra(input)
#idea is to relax past a edge and see if leads to reduce time
total=dist[endx,endy]

cheatspace=defaultdict(lambda: [])

for i in range(-20,21):
    for j in range(-20,21):
        if abs(i) + abs(j) <= 20 and (abs(i) >= 2 or abs(j) >=2):
            cheatspace[abs(i) + abs(j)].append((i,j))


def part1():
    shortcut=defaultdict(lambda : 0)
    for cord in orginal_path:
        ci,cj=cord
        for dir in directions:
            i,j=dir
            i*=2
            j*=2
            di,dj=ci+i,cj+j
            if (di,dj) not in dist: #another wall
                continue
            if dist[ci,cj]+2 < dist[di,dj]:
                shortcut[(dist[di,dj] - (dist[ci,cj]+2))]+=1
    cheats=0
    for key in shortcut:
        if key >= 100:
            cheats+=shortcut[key]
    print(cheats)


def calculateCord(cord,i,j):
    ci,cj=cord
    ci=i+ci
    cj=j+cj
    return ci,cj

def part2():
    cheats=defaultdict(lambda: 0)
    def bestincheatspace(cord):
        ci,cj=cord
        for key in cheatspace:
            for point in cheatspace[key]:
                i,j=point
                di,dj = calculateCord(cord,i,j)
                if (di,dj) not in dist: #another wall
                    continue
                if dist[ci,cj]+key < dist[di,dj]:
                    cheats[(dist[di,dj] - (dist[ci,cj]+key))]+=1
    for cord in orginal_path:
        bestincheatspace(cord)
    keys = sorted(cheats.keys(),reverse=True)
    part2=0
    for cheat in keys:
        if cheat>=100:
            part2+=cheats[cheat]
        else:
            break
    print(part2)


part1()
part2()

#############################################################################################################################################
#.......###...###...#.....#...#.....#...#.....#...#...#.....#.........#...###...#...###.........#.......###...#...###.........#.....#.......#
#.#####.###.#.###.#.#.###.#.#.#.###.#.#.#.###.#.#.#.#.#.###.#.#######.#.#.###.#.#.#.###.#######.#.#####.###.#.#.#.###.#######.#.###.#.#####.#
#.#...#.....#.#...#.#.#...#.#.#...#.#.#.#...#.#.#.#.#.#.#...#.......#...#...#.#.#.#...#.....#...#...#...#...#...#...#.......#.#.#...#.#.....#
#.#.#.#######.#.###.#.#.###.#.###.#.#.#.###.#.#.#.#.#.#.#.#########.#######.#.#.#.###.#####.#.#####.#.###.#########.#######.#.#.#.###.#.#####
#...#.......#.#...#...#.#...#.#...#.#.#.#...#.#.#.#.#.#.#.......###.....#...#.#.#...#.#...#.#...###.#...#...#.......#...###.#.#.#.#...#.....#
###########.#.###.#####.#.###.#.###.#.#.#.###.#.#.#.#.#.#######.#######.#.###.#.###.#.#.#.#.###.###.###.###.#.#######.#.###.#.#.#.#.#######.#
#.....